In [3]:
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [4]:
def get_stock_data(ticker, start_date, end_date):
    data = yf.download(ticker, start=start_date, end=end_date)
    return data

def calculate_beta(ticker, start_date, end_date):
    tickers = [ticker, "SPY"]
    covariance = calculate_covariance(tickers, start_date, end_date)
    variance = calculate_variance("SPY", start_date, end_date)
    beta = covariance / variance
    return beta

#CAPM model
def calculate_return(ticker, start_date, end_date):
    risk_free_rate = 4.58   #source: https://www.cnbc.com/quotes/US10Y
    beta = calculate_beta(ticker, start_date, end_date)
    beta = beta[ticker]["SPY"]
    expected_market_return = 8 #source: https://www.cnbc.com/quotes/US10Y
    return risk_free_rate + beta * (expected_market_return - risk_free_rate)

def calculate_variance(ticker, start_date, end_date):
    stock_data = get_stock_data(ticker, start_date, end_date)
    stock_data = stock_data['Close'][ticker].pct_change().dropna()
    return stock_data.var() 


def calculate_covariance(tickers, start_date, end_date):
    data = {}  

    for ticker in tickers:
        
        stock_data = get_stock_data(ticker, start_date, end_date)['Close'][ticker].pct_change().dropna()
        
        if ticker not in data:
            data[ticker] = []
        
        
        for ret in stock_data:
            data[ticker].append(ret)
            
        
    df = pd.DataFrame(data)

    return df.cov()

In [11]:
import cvxpy as cp


def optimize_portfolio(tickers, start_date, end_date):
    expected_returns_dict = {}
    expected_returns = []

    for ticker in tickers:
        expected_returns_dict[calculate_return(ticker, start_date, end_date)] = ticker
        expected_returns.append(calculate_return(ticker, start_date, end_date))

    cov_matrix = calculate_covariance(tickers, start_date, end_date)
    n = 10
    w = cp.Variable(n)
    portfolio_variance = cp.quad_form(w, cov_matrix)
    objective = cp.Minimize(portfolio_variance)


    



In [ ]:
nasdaq =["AAPL", "ABNB", "ADBE", "ADI", "ADP", "ADSK", "AEP", "AMAT", "AMD", "AMGN", 
"AMZN", "ANSS", "APP", "ASML", "AVGO", "AXON", "AZN", "BKR", "CDNS", 
"CEG", "CHTR", "CMCSA", "CSGP", "CSX", "CTAS", "CTSH", "DDOG", "DLTR", "DXCM", 
"EA", "EBAY", "EXC", "FAST", "FANG", "FTNT", "GFS", "GILD", "HON", "IDXX", 
"ILMN", "INTC", "INTU", "ISRG", "JD", "KDP", "KLAC", "LCID", "LRCX", "MAR", 
"MCHP", "MDLZ", "MELI", "META", "MNST", "MRVL", "MSFT", "MU", "NFLX", "NTES", 
"NVDA", "NXPI", "ORLY", "PANW", "PCAR", "PDD", "PEP", "PYPL", "QCOM", "REGN", 
"ROST", "SBUX", "SNPS", "SWKS", "TEAM", "TMUS", "TSLA", "TTD", "TTWO", 
"TXN", "VRSK", "VRTX", "WBD", "WDAY", "XEL", "ZS"]

optimize_portfolio(nasdaq[:10], "2024-01-01", "2024-12-01")